In [ ]:
import paddle
import paddle.fluid as fluid
import numpy as np
import sys
import math

CLASS_DIM = 2     # positive adn negative
EMB_DIM = 128     # dimension of word embedding, one character
HID_DIM = 512     # dimension of hidden layer
STACKED_NUM = 3   # layers of bidirectional stack
BATCH_SIZE = 128 

## Model

word embedding
https://keras.io/zh/layers/embeddings/

### TEXT CNN

In [ ]:
def convolution_net(data, input_dim, class_dim, emb_dim, hid_dim):
    emb = fluid.embedding(input=data, # data: word
                          size=[input_dim, emb_dim],  # input_dim: total num of words; emb_dim: dimension of word embdding
                          is_sparse=True)  # save RAM

    conv_3 = fluid.nets.sequence_conv_pool(
        input=emb,
        num_filters=hid_dim,  # num of convolution kernels
        filter_size=3,  # size of kernel: 3*3 or 5*5
        act="tanh",
        pool_type="sqrt")

    # second conv which doesn't use the output from the above layer
    conv_4 = fluid.nets.sequence_conv_pool(
        input=emb,
        num_filters=hid_dim,
        filter_size=4,
        act="tanh",
        pool_type="sqrt")
    # conv_3 and conv_4 use different kernel

    prediction = fluid.layers.fc(
        input=[conv_3, conv_4],
        size=class_dim, # positive or negative
        act="softmax") # softmax

    return prediction

### Stacked Bidrectional LSTM

![](https://githubraw.cdn.bcebos.com/PaddlePaddle/book/develop/06.understand_sentiment/image/stacked_lstm.jpg?raw=true)

In [ ]:
def stacked_lstm_net(data, input_dim, class_dim, emb_dim, hid_dim, stacked_num):

    emb = fluid.embedding(
        input=data, size=[input_dim, emb_dim], is_sparse=True)

    # the first layer
    # fc
    fc1 = fluid.layers.fc(input=emb, size=hid_dim)
    # lstm
    lstm1, cell1 = fluid.layers.dynamic_lstm(input=fc1, size=hid_dim)

    inputs = [fc1, lstm1] # combine to one list

    # the rest layers
    for i in range(2, stacked_num + 1):
        fc = fluid.layers.fc(input=inputs, size=hid_dim)
        lstm, cell = fluid.layers.dynamic_lstm(input=fc, size=hid_dim, is_reverse=(i % 2) == 0)
        inputs = [fc, lstm]

    # pooling
    fc_last = fluid.layers.sequence_pool(input=inputs[0], pool_type='max') # inputs[0]: outputs from fc
    lstm_last = fluid.layers.sequence_pool(input=inputs[1], pool_type='max') # inputs[1]: outputs from LSTM

    # fc: softmax prediction
    prediction = fluid.layers.fc(input=[fc_last, lstm_last], size=class_dim, act='softmax')

    return prediction

## Inference program

In [ ]:
def inference_program(word_dict):
    data = fluid.data(
        name="words", shape=[None], dtype="int64", lod_level=1) # 词，one-hot
    dict_dim = len(word_dict) # len of words, ~ 5000
#     prediction = convolution_net(data, dict_dim, CLASS_DIM, EMB_DIM, HID_DIM)
    prediction = stacked_lstm_net(data, dict_dim, CLASS_DIM, EMB_DIM, HID_DIM, STACKED_NUM)

    return prediction

## Average loss, accuracy and optimizer

In [ ]:
def train_program(prediction):
    label = fluid.data(name="label", shape=[None, 1], dtype="int64")
    cost = fluid.layers.cross_entropy(input=prediction, label=label)
    avg_cost = fluid.layers.mean(cost)
    accuracy = fluid.layers.accuracy(input=prediction, label=label)
    return [avg_cost, accuracy]

def optimizer_func():
    return fluid.optimizer.Adagrad(learning_rate=0.002)

In [ ]:
use_cuda = False
place = fluid.CUDAPlace(0) if use_cuda else fluid.CPUPlace()

## Load data

In [ ]:
print("Loading IMDB word dict....")
word_dict = paddle.dataset.imdb.word_dict()

print ("Reading training data....")
train_reader = fluid.io.batch(
    fluid.io.shuffle(
        paddle.dataset.imdb.train(word_dict), buf_size=25000),
    batch_size=BATCH_SIZE)

print("Reading testing data....")
test_reader = fluid.io.batch(
    paddle.dataset.imdb.test(word_dict), batch_size=BATCH_SIZE)

## Executor

In [ ]:
exe = fluid.Executor(place)

prediction = inference_program(word_dict)
[avg_cost, accuracy] = train_program(prediction)

sgd_optimizer = optimizer_func()
_ = sgd_optimizer.minimize(avg_cost)

## Validation

In [ ]:
def train_test(program, reader):
    count = 0
    feed_var_list = [
        program.global_block().var(var_name) for var_name in feed_order
    ]
    feeder_test = fluid.DataFeeder(feed_list=feed_var_list, place=place)
    test_exe = fluid.Executor(place)
    accumulated = len([avg_cost, accuracy]) * [0]
    for test_data in reader():
        avg_cost_np = test_exe.run(
            program=program,
            feed=feeder_test.feed(test_data),
            fetch_list=[avg_cost, accuracy])
        accumulated = [
            x[0] + x[1][0] for x in zip(accumulated, avg_cost_np)
        ]
        count += 1

    return [x / count for x in accumulated]

## Main loop

In [ ]:
# Specify the directory path to save the parameters
params_dirname = "understand_sentiment_conv.inference.model"

feed_order = ['words', 'label']
pass_num = 1


def train_loop(main_program):
    exe.run(fluid.default_startup_program()) # Initialization

    feed_var_list_loop = [main_program.global_block().var(var_name)
                          for var_name in feed_order]
    feeder = fluid.DataFeeder(feed_list=feed_var_list_loop, place=place)

    test_program = fluid.default_main_program().clone(for_test=True)

    # training
    for epoch_id in range(pass_num):
        for step_id, data in enumerate(train_reader()): # get data from train_reader()
            metrics = exe.run(main_program,
                              feed=feeder.feed(data),
                              fetch_list=[avg_cost, accuracy])

            # test results
            avg_cost_test, acc_test = train_test(test_program, test_reader)
            print('Step {0}, Test Loss {1:0.2}, Acc {2:0.2}'.format(
                step_id, avg_cost_test, acc_test))

            print("Step {0}, Epoch {1} Metrics {2}".format(
                step_id, epoch_id, list(map(np.array,
                                            metrics))))

            if step_id == 30:
                if params_dirname is not None:
                    fluid.io.save_inference_model(params_dirname, ["words"],
                                                  prediction, exe)
                return

In [ ]:
train_loop(fluid.default_main_program())

### Application

In [ ]:
place = fluid.CUDAPlace(0) if use_cuda else fluid.CPUPlace()
exe = fluid.Executor(place)
inference_scope = fluid.core.Scope()

In [ ]:
reviews_str = [
    b'read the book forget the movie', b'this is a great movie', b'this is very bad'
]
reviews = [c.split() for c in reviews_str]

UNK = word_dict['<unk>']
lod = []
for c in reviews:
    lod.append([word_dict.get(words, UNK) for words in c])

base_shape = [[len(c) for c in lod]]
lod = np.array(sum(lod, []), dtype=np.int64)

tensor_words = fluid.create_lod_tensor(lod, base_shape, place)

In [ ]:
with fluid.scope_guard(inference_scope):

    [inferencer, feed_target_names,
     fetch_targets] = fluid.io.load_inference_model(params_dirname, exe)

    assert feed_target_names[0] == "words"
    results = exe.run(inferencer,
                      feed={feed_target_names[0]: tensor_words},
                      fetch_list=fetch_targets,
                      return_numpy=False)
    np_data = np.array(results[0])
    for i, r in enumerate(np_data):
        print("Predict probability of ", r[0], " to be positive and ", r[1],
              " to be negative for review \'", reviews_str[i], "\'")